In [ ]:
import requests
import datetime
from datetime import date
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm 

In [ ]:
pd.set_option('display.max_columns', None)
df1=pd.read_csv('C:/Users/HHHH/df1.csv')
df2=pd.read_csv('C:/Users/HHHH/df2.csv')
df3=pd.read_csv('C:/Users/HHHH/df3.csv')
df4=pd.read_csv('C:/Users/HHHH/df4.csv')
df = df1.append(df2).append(df3).append(df4)
df.shape

In [ ]:
cols = [0,1,5,9,10,42,43,44,45,46]
df = df.drop(df.columns[cols], axis =1).dropna(subset=['Value '])
df = df[(df['Preferred Positions '] != 'GK')].reset_index(drop=True)
df['Height '] = df['Height '].map(lambda x: x.split(' ')[0]).astype(int)
df['Weight '] = df['Weight '].map(lambda x: x.split(' ')[0]).astype(int)
df['BMI'] = df['Weight '] / (df['Height ']*df['Height ']) * 10000
df['BMI'] = df['BMI'].astype(int)
df['Value '] = df['Value '].str.replace(".", "").str.replace("£", "").astype(int)
df['Wage '] = df['Wage '].str.replace(".", "").str.replace("£", "").astype(int)
df['Deal'] = df['Value '] / df['Wage ']
df['Deal'] = df['Deal'].astype(int)
df = df.reset_index(drop=True)
print(df.shape)
df.tail(2)

In [ ]:
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, make_scorer


### put everything together

In [ ]:
X, y = df.drop(['Value ','Wage ','Deal'], axis=1), df['Value ']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

numerical_ix = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X_train.select_dtypes(include=['object', 'bool']).columns

t = [('cat', ce.OneHotEncoder(use_cat_names=True), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)

model = RandomForestRegressor()
#model = GradientBoostingRegressor()
pipeline = Pipeline(steps=[('prep',col_transform), ('m', model)])

cv = KFold(n_splits=5, shuffle=True, random_state=0)

grid_param = { 
            "m__n_estimators"      : [100, 300, 500],
            "m__max_features"      : ["auto", "sqrt", "log2"],
            "m__min_samples_split" : [2,4,8],
            "m__bootstrap": [True, False],
            }


gd_sr = GridSearchCV(pipeline,
                     param_grid=grid_param,
                     #scoring = make_scorer(mean_squared_error),
                     scoring='r2',
                     cv=cv,
                     n_jobs=3)

# This is for GradientBoostRegressor
# param_grid={'m__n_estimators':[100,200,500],
#             'm__max_depth': [4, 6, 8]} 

# gd_sr = GridSearchCV(pipeline,
#                      param_grid=param_grid,
#                      #scoring = make_scorer(mean_squared_error),
#                      scoring='r2',
#                      cv=5,#equivalent of setting it to KFold(n_splits=5)
#                      n_jobs=1)


gd_sr.fit(X_train, y_train)

print(gd_sr.best_params_)
print(gd_sr.best_score_)
print(gd_sr.best_estimator_)

In [ ]:
y_pred = gd_sr.predict(X_test)
from sklearn.metrics import r2_score
test_score = r2_score(y_test, y_pred)
print(test_score)


### get the feature importance

In [ ]:
pipeline.fit(X_train,y_train)
feature_name = pipeline.named_steps['prep'].transformers_[0][1].get_feature_names()

num_name = numerical_ix.tolist()
for i in range(len(num_name)):
    feature_name.append(num_name[i])
    i=i+1

feature_importance = gd_sr.best_estimator_.named_steps['m'].feature_importances_
output = pd.DataFrame(list(zip(feature_name,feature_importance)))
output.columns = ['feature_name','feature_importance']
output.sort_values(by='feature_importance', ascending=False)


In [ ]:

plt.figure(figsize=(16, 6))
plt.yscale('log', nonposy='clip')
plt.bar(feature_name, feature_importance, align='center')
#plt.bar(feature_name, feature_name, align='center')
#plt.xticks(range(len(feature_importance)), features, rotation='vertical')
#plt.title('Feature importance')
#plt.ylabel('Importance')
#plt.xlabel('Features')
plt.show()